<a href="https://colab.research.google.com/github/ekansh09/IIITH-HAI-Assignment-2/blob/main/CVC_Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
import tifffile as tiff
from skimage.transform import resize


## CVC-ClinicDB Dataset

Upload Kaggle.json

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d balraj98/cvcclinicdb

 99% 130M/131M [00:01<00:00, 68.2MB/s]
100% 131M/131M [00:01<00:00, 70.4MB/s]


In [ ]:
!unzip /content/cvcclinicdb.zip

In [ ]:
img_files = next(os.walk('/content/PNG/Original'))[2]
msk_files = next(os.walk('/content/PNG/Ground Truth'))[2]

img_files.sort()
msk_files.sort()

## Pre-Processing

In [ ]:
X_temp = []
Y_temp = []

for img_fl in tqdm(img_files):    
    if(img_fl.split('.')[-1]=='png'):
        img = cv2.imread('/content/PNG/Original/'+ str(img_fl), cv2.IMREAD_COLOR)
        resized_img = cv2.resize(img,(192, 256), interpolation = cv2.INTER_CUBIC)
        X_temp.append(resized_img)

100%|██████████| 612/612 [00:01<00:00, 329.37it/s]


In [ ]:
for msk_fl in tqdm(msk_files):    
    if(msk_fl.split('.')[-1]=='png'):
        img = cv2.imread('/content/PNG/Ground Truth/'+ str(msk_fl), cv2.IMREAD_GRAYSCALE)
        resized_img = cv2.resize(img,(192, 256), interpolation = cv2.INTER_CUBIC)
        Y_temp.append(resized_img)

100%|██████████| 612/612 [00:00<00:00, 826.46it/s]


In [ ]:
print(len(X_temp))
print(len(Y_temp))

X = np.array(X_temp)
Y = np.array(Y_temp)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

Y_train = Y_train.reshape((Y_train.shape[0],Y_train.shape[1],Y_train.shape[2],1))
Y_test = Y_test.reshape((Y_test.shape[0],Y_test.shape[1],Y_test.shape[2],1))

X_train = X_train / 255
X_test = X_test / 255
Y_train = Y_train / 255
Y_test = Y_test / 255


Y_train = np.round(Y_train,0)	
Y_test = np.round(Y_test,0)	

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

612
612
(489, 256, 192, 3)
(489, 256, 192, 1)
(123, 256, 192, 3)
(123, 256, 192, 1)


## MultiResUNet


In [ ]:
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False)(x)
    x = BatchNormalization( scale=False)(x)
    if(activation == None):
        return x
    x = Activation(activation, name=name)(x)
    return x

In [ ]:
def trans_conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(2, 2)):
    x = Conv2DTranspose(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = BatchNormalization(scale=False)(x)
    return x

In [ ]:
def MultiResBlock(U, inp, alpha = 1.67):

    W = alpha * U
    shortcut = inp

    shortcut = conv2d_bn(shortcut, int(W*0.167) + int(W*0.333) +  int(W*0.5), 1, 1, activation=None, padding='same')
    conv3x3 = conv2d_bn(inp, int(W*0.167), 3, 3, activation='relu', padding='same')
    conv5x5 = conv2d_bn(conv3x3, int(W*0.333), 3, 3, activation='relu', padding='same')
    conv7x7 = conv2d_bn(conv5x5, int(W*0.5), 3, 3, activation='relu', padding='same')
    out = concatenate([conv3x3, conv5x5, conv7x7], axis=3)
    out = BatchNormalization()(out)
    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization()(out)

    return out


In [ ]:
def ResPath(filters, length, inp):
    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1, activation=None, padding='same')
    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')
    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization()(out)

    for i in range(length-1):
        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1, activation=None, padding='same')
        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')
        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = BatchNormalization()(out)

    return out

In [ ]:
def MultiResUnet(height, width, n_channels):

    inputs = Input((height, width, n_channels))
    
    mresblock1 = MultiResBlock(32, inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(mresblock1)
    mresblock1 = ResPath(32, 4, mresblock1)

    mresblock2 = MultiResBlock(32*2, pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(mresblock2)
    mresblock2 = ResPath(32*2, 3, mresblock2)

    mresblock3 = MultiResBlock(32*4, pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(mresblock3)
    mresblock3 = ResPath(32*4, 2, mresblock3)

    mresblock4 = MultiResBlock(32*8, pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(mresblock4)
    mresblock4 = ResPath(32*8, 1, mresblock4)

    mresblock5 = MultiResBlock(32*16, pool4)

    up6 = concatenate([trans_conv2d_bn(mresblock5,32*8, 2, 2), mresblock4])
    mresblock6 = MultiResBlock(32*8, up6)

    up7 = concatenate([trans_conv2d_bn(mresblock6, 32*4, 2, 2), mresblock3])
    mresblock7 = MultiResBlock(32*4, up7)

    up8 = concatenate([trans_conv2d_bn(mresblock7, 32*2, 2, 2), mresblock2])
    mresblock8 = MultiResBlock(32*2, up8)

    up9 = concatenate([trans_conv2d_bn(mresblock8, 32, 2, 2), mresblock1])
    mresblock9 = MultiResBlock(32, up9)

    conv10 = conv2d_bn(mresblock9, 1, 1, 1, activation='sigmoid')
    
    
    model = Model(inputs=[inputs], outputs=[conv10])

                  
    return model

###Evaluation Metrics

In [ ]:
def dice_coef(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
def jacard(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum ( y_true_f * y_pred_f)
    union = K.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)
    return intersection/union

####Save Model

In [ ]:
def saveModel(model):

    model_json = model.to_json()

    try:
        os.makedirs('models')
    except:
        pass
    
    fp = open('models/modelP.json','w')
    fp.write(model_json)
    model.save_weights('models/modelW.h5')

####Evaluate Model

In [ ]:
def evaluateModel(model, X_test, Y_test, batchSize):
    
    try:
        os.makedirs('results')
    except:
        pass 
    

    yp = model.predict(x=X_test,batch_size=batchSize, verbose=1)
    print('Min : '+ str(np.min(yp)))
    print('Max : '+ str(np.max(yp)))
    yp = np.round_(yp,0)
    print('Unique after rounding : '+ str(np.unique(yp)))
    # print(yp)

    for i in range(4):
        
        plt.figure(figsize=(20,4))
        plt.subplot(1,3,1)
        plt.imshow(X_test[i], cmap='gray')
        plt.title('Input')
        plt.subplot(1,3,2)
        plt.imshow(Y_test[i].reshape(Y_test[i].shape[0],Y_test[i].shape[1])*255, cmap='gray')
        plt.title('Ground Truth')
        plt.subplot(1,3,3)
        plt.imshow(yp[i].reshape(yp[i].shape[0],yp[i].shape[1])*255, cmap='gray')
        plt.title('Prediction')
        intersection1 = yp[i].ravel() * Y_test[i].ravel()
        # plt.show()
        union1 = yp[i].ravel() + Y_test[i].ravel() - intersection1
        # plt.show()

        jacard = (np.sum(intersection1)/np.sum(union1))  
        plt.suptitle('Jacard Index'+ str(np.sum(intersection1)) +'/'+ str(np.sum(union1)) +'='+str(jacard))

        plt.savefig('results/'+str(i)+'.png',format='png')
        plt.show()

        plt.close()


    jacard = 0
    dice = 0
    
    
    for i in range(len(Y_test)):
        yp_2 = yp[i].ravel()
        y2 = Y_test[i].ravel()
        
        intersection = yp_2 * y2
        union = yp_2 + y2 - intersection

        jacard += (np.sum(intersection)/np.sum(union))  

        dice += (2. * np.sum(intersection) + 1) / (np.sum(yp_2) + np.sum(y2) + 1)

    
    jacard /= len(Y_test)
    dice /= len(Y_test)
    


    print('Jacard Index : '+str(jacard))
    print('Dice Coefficient : '+str(dice))
    

    fp = open('models/log.txt','a')
    fp.write(str(jacard)+'\n')
    fp.close()

    fp = open('models/best.txt','r')
    best = fp.read()
    fp.close()

    if(jacard>float(best)):
        print('***********************************************')
        print('Jacard Index improved from '+str(best)+' to '+str(jacard))
        print('***********************************************')
        fp = open('models/best.txt','w')
        fp.write(str(jacard))
        fp.close()

        saveModel(model)

####Training

In [ ]:
def trainStep(model, X_train, Y_train, X_test, Y_test, epochs, batchSize):

    for epoch in range(epochs):
      print('Epoch : {}'.format(epoch+1))
      model.fit(X_train, Y_train, epochs=1, verbose=1, batch_size=batchSize)     
      evaluateModel(model,X_test, Y_test, batchSize)
    return model

##UNet++

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Concatenate, UpSampling2D, Input
import random
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *

In [ ]:

dropout_rate = 0.5
act = "relu"

def standard_unit(input_tensor, nb_filter, kernel_size=3):

    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(input_tensor)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(dropout_rate)(x)

    return x

def UNetPlusPlus(input_size):

    nb_filter = [32,64,128,256,512]

    img_input = Input(input_size)



    conv1_1 = standard_unit(img_input, nb_filter=nb_filter[0])
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_1)

    conv2_1 = standard_unit(pool1,nb_filter=nb_filter[1])
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2_1)

    up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), padding='same')(conv2_1)
    conv1_2 = concatenate([up1_2, conv1_1])
    conv1_2 = standard_unit(conv1_2, nb_filter=nb_filter[0])

    conv3_1 = standard_unit(pool2, nb_filter=nb_filter[2])
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3_1)

    up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, conv2_1])
    conv2_2 = standard_unit(conv2_2, nb_filter=nb_filter[1])

    up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, conv1_1, conv1_2])
    conv1_3 = standard_unit(conv1_3, nb_filter=nb_filter[0])

    conv4_1 = standard_unit(pool3, nb_filter=nb_filter[3])
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4_1)

    up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1])
    conv3_2 = standard_unit(conv3_2, nb_filter=nb_filter[2])

    up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, conv2_1, conv2_2])
    conv2_3 = standard_unit(conv2_3, nb_filter=nb_filter[1])

    up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3])
    conv1_4 = standard_unit(conv1_4, nb_filter=nb_filter[0])

    conv5_1 = standard_unit(pool4, nb_filter=nb_filter[4])

    up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1])
    conv4_2 = standard_unit(conv4_2, nb_filter=nb_filter[3])

    up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2])
    conv3_3 = standard_unit(conv3_3, nb_filter=nb_filter[2])

    up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3])
    conv2_4 = standard_unit(conv2_4, nb_filter=nb_filter[1])

    up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4])
    conv1_5 = standard_unit(conv1_5, nb_filter=nb_filter[0])

    nestnet_output_4 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_5)
 
    model = Model(inputs=img_input, outputs=[nestnet_output_4])

    return model


##UNet

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """ Bridge """
    b1 = conv_block(p4, 1024)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    """ Outputs """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    """ Model """
    model = Model(inputs, outputs)
    return model

##Models

In [ ]:
# model = UNetPlusPlus((256,256,3))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef, jacard, 'accuracy'])

# saveModel(model)
# model.summary()   

In [ ]:
model = MultiResUnet(height=256, width=192, n_channels=3)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef, jacard, 'accuracy'])

saveModel(model)
model.summary()   

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 192, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 192, 8)  216         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 192, 8)  24          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 192, 8)  0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

In [ ]:
# model = build_unet((256,256,3))
# model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[dice_coef, jacard, 'accuracy'])
# saveModel(model)
# model.summary()      

#####Logs Saving

In [ ]:
fp = open('models/log.txt','w')
fp.close()
fp = open('models/best.txt','w')
fp.write('-1.0')
fp.close()

####Running Model

In [ ]:
with tf.device('/device:GPU:0'):
  trainStep(model , X_train, Y_train, X_test, Y_test, epochs=250, batchSize = 10 )